对基准模型的超参数进行优化

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取数据
file_path = "D:\Pythonworkshop\Solar_data_processing\cleaned_data_final_Yulara.csv"
data = pd.read_csv(file_path)

# 提取特征和目标变量
X = data.drop(columns=['Active_Power','timestamp'])
y = data['Active_Power']

# 按照70%、15%、15%划分训练集、验证集和测试集
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("数据集划分完成：")
print(f"训练集大小: {X_train.shape}")
print(f"验证集大小: {X_val.shape}")
print(f"测试集大小: {X_test.shape}")


<>:5: SyntaxWarning: invalid escape sequence '\P'
<>:5: SyntaxWarning: invalid escape sequence '\P'
C:\Users\bo.chen18\AppData\Local\Temp\ipykernel_19692\1741114340.py:5: SyntaxWarning: invalid escape sequence '\P'
  file_path = "D:\Pythonworkshop\Solar_data_processing\cleaned_data_final_Yulara.csv"


数据集划分完成：
训练集大小: (73040, 10)
验证集大小: (15651, 10)
测试集大小: (15652, 10)


In [2]:
import numpy as np

# 将特征和目标变量按照序列移动，形成单步预测
def create_sequences(X, y, n_steps):
    Xs, ys = [], []
    for i in range(len(X) - n_steps):
        Xs.append(X.iloc[i:(i + n_steps)].values)
        ys.append(y.iloc[i + n_steps])
    return np.array(Xs), np.array(ys)

# 使用 n_steps = 1 (单步预测)
n_steps = 1
X_train_seq, y_train_seq = create_sequences(X_train, y_train, n_steps)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, n_steps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, n_steps)


In [13]:
from sklearn.neural_network import MLPRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical

# 定义MLP的超参数搜索空间
mlp_search_space = {
    'hidden_layer_sizes': Categorical([(50,), (100,)]),
    'activation': Categorical(['relu']),
    'solver': Categorical(['adam']),
    'alpha': Real(1e-5, 1e-2, prior='log-uniform'),  # 调整alpha的范围，确保数值在合理范围内
    'learning_rate': Categorical(['constant']),
    'learning_rate_init': Real(1e-3, 1e-1, prior='log-uniform')  # 调整learning_rate_init的范围
}


# 初始化MLP模型
mlp = MLPRegressor(max_iter=1000, random_state=42)

# 使用贝叶斯优化进行超参数搜索
mlp_opt = BayesSearchCV(estimator=mlp, search_spaces=mlp_search_space, n_iter=32, cv=5, random_state=42, n_jobs=-1)

# 进行搜索并获取最佳参数
mlp_opt.fit(X_train, y_train)
print("Best parameters found for MLP:", mlp_opt.best_params_)
print("Best score for MLP:", mlp_opt.best_score_)


ValueError: Not all points are within the bounds of the space.

In [ ]:
from sklearn.svm import SVR

# 定义SVM的超参数搜索空间
svm_search_space = {
    'C': Real(1e-6, 1e+2, prior='log-uniform'),
    'epsilon': Real(1e-6, 1e+1, prior='log-uniform'),
    'kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
    'degree': Integer(1, 5),  # 适用于'poly' kernel
    'gamma': Categorical(['scale', 'auto']),
}

# 初始化SVM模型
svm = SVR()

# 使用贝叶斯优化进行超参数搜索
svm_opt = BayesSearchCV(estimator=svm, search_spaces=svm_search_space, n_iter=32, cv=5, random_state=42, n_jobs=-1)

# 进行搜索并获取最佳参数
svm_opt.fit(X_train, y_train)
print("Best parameters found for SVM:", svm_opt.best_params_)
print("Best score for SVM:", svm_opt.best_score_)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras_tuner import BayesianOptimization

def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=16), 
                   activation='relu', 
                   input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='mse')
    return model

tuner = BayesianOptimization(
    build_lstm_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='lstm_tuner',
    project_name='lstm_bayes_opt'
)

# 执行贝叶斯优化
tuner.search(X_train_seq, y_train_seq, epochs=10, validation_data=(X_val_seq, y_val_seq))

# 获取最优模型
best_lstm_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters for LSTM:", best_hyperparameters.values)


In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

def build_cnn_lstm_model(hp):
    model = Sequential()
    model.add(Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=16), 
                     kernel_size=hp.Int('kernel_size', min_value=1, max_value=5), 
                     activation='relu', 
                     input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='mse')
    return model

tuner = BayesianOptimization(
    build_cnn_lstm_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='cnn_lstm_tuner',
    project_name='cnn_lstm_bayes_opt'
)

# 执行贝叶斯优化
tuner.search(X_train_seq, y_train_seq, epochs=10, validation_data=(X_val_seq, y_val_seq))

# 获取最优模型
best_cnn_lstm_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters_cnn_lstm = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters for CNN-LSTM:", best_hyperparameters_cnn_lstm.values)


In [ ]:
# 使用最优参数训练MLP
best_mlp = mlp_opt.best_estimator_
best_mlp.fit(X_train, y_train)
y_pred_mlp = best_mlp.predict(X_test)

# 使用最优参数训练SVM
best_svm = svm_opt.best_estimator_
best_svm.fit(X_train, y_train)
y_pred_svm = best_svm.predict(X_test)

# 使用最优LSTM模型进行预测
y_pred_lstm = best_lstm_model.predict(X_test_seq).flatten()

# 使用最优CNN-LSTM模型进行预测
y_pred_cnn_lstm = best_cnn_lstm_model.predict(X_test_seq).flatten()

# 评估优化后的模型
mae_mlp, mse_mlp, rmse_mlp, nrmse_mlp = evaluate_model(y_test, y_pred_mlp)
mae_svm, mse_svm, rmse_svm, nrmse_svm = evaluate_model(y_test, y_pred_svm)
mae_lstm, mse_lstm, rmse_lstm, nrmse_lstm = evaluate_model(y_test_seq, y_pred_lstm)
mae_cnn_lstm, mse_cnn_lstm, rmse_cnn_lstm, nrmse_cnn_lstm = evaluate_model(y_test_seq, y_pred_cnn_lstm)

print(f"Optimized MLP - MAE: {mae_mlp}, MSE: {mse_mlp}, RMSE: {rmse_mlp}, NRMSE: {nrmse_mlp}")
print(f"Optimized SVM - MAE: {mae_svm}, MSE: {mse_svm}, RMSE: {rmse_svm}, NRMSE: {nrmse_svm}")
print(f"Optimized LSTM - MAE: {mae_lstm}, MSE: {mse_lstm}, RMSE: {rmse_lstm}, NRMSE: {nrmse_lstm}")
print(f"Optimized CNN-LSTM - MAE: {mae_cnn_lstm}, MSE: {mse_cnn_lstm}, RMSE: {rmse_cnn_lstm}, NRMSE: {nrmse_cnn_lstm}")
